In [18]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D

In [19]:
PATH = "./shapes/"
IMG_SIZE = 64
shapes = ["circle", "square", "triangle", "star"]
labels = []
dataset = []
for shape in shapes:
    print("Getting data for: ", shape)
    for path in os.listdir(PATH + shape):
        #add the image to the list of images
        image = cv2.imread(PATH + shape + '/' + path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        dataset.append(image)
        #add an integer to the labels list 
        labels.append(shapes.index(shape))

print("\nDataset Images size:", len(dataset))
print("Image Shape:", dataset[0].shape)
print("Labels size:", len(labels))

Getting data for:  circle
Getting data for:  square
Getting data for:  triangle
Getting data for:  star

Dataset Images size: 14970
Image Shape: (64, 64, 3)
Labels size: 14970


In [20]:
data = np.array(dataset)
data = data.astype("float32")/255.0

#one_hot_encode labels
output = np.array(labels)
output = to_categorical(output)

In [21]:
x_train,x_test,y_train,y_test = train_test_split(data, output,test_size = 0.2,random_state = 0)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((11976, 64, 64, 3), (11976, 4), (2994, 64, 64, 3), (2994, 4))

In [26]:
def nn(rows,cols,channels,activation):
    model = Sequential()
    shape = (rows,cols,channels)
    model.add(Conv2D(filters = 6,kernel_size = 5,input_shape = shape))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size = (2,2),strides = (2,2)))
    
    model.add(Conv2D(filters = 16,kernel_size = 5))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size = (2,2),strides = (2,2)))
    
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dropout(0.5))
    
    model.add(Dense(4))
    model.add(Activation("softmax"))
    
    return model

In [23]:
bs = 120
rate = 0.01
epoch = 10
opt = SGD(lr = rate)


In [27]:
model = nn(64,64,3,"relu")
model.compile(loss = "categorical_crossentropy",optimizer = opt,metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 60, 60, 6)         456       
_________________________________________________________________
activation_9 (Activation)    (None, 60, 60, 6)         0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 30, 30, 6)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 26, 16)        2416      
_________________________________________________________________
activation_10 (Activation)   (None, 26, 26, 16)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2704)              0         
__________

In [29]:
try1 = model.fit(x_train,y_train,validation_data = (x_test,y_test), batch_size = bs,epochs = epoch,verbose=1)
scores_train = model.evaluate(x_train,y_train,verbose = 1)
scores_test = model.evaluate(x_test,y_test,verbose = 1)
scores_train[1], scores_test[1]

Train on 11976 samples, validate on 2994 samples
Epoch 1/10
11976/11976 [==============================] - 31s 3ms/step - loss: 0.0085 - acc: 0.9982 - val_loss: 0.0032 - val_acc: 0.9990
Epoch 2/10
11976/11976 [==============================] - 32s 3ms/step - loss: 0.0063 - acc: 0.9989 - val_loss: 0.0022 - val_acc: 0.9997
Epoch 3/10
11976/11976 [==============================] - 30s 3ms/step - loss: 0.0056 - acc: 0.9990 - val_loss: 0.0018 - val_acc: 0.9997
Epoch 4/10
11976/11976 [==============================] - 30s 3ms/step - loss: 0.0052 - acc: 0.9989 - val_loss: 0.0016 - val_acc: 0.9997
Epoch 5/10
11976/11976 [==============================] - 30s 3ms/step - loss: 0.0047 - acc: 0.9992 - val_loss: 0.0016 - val_acc: 0.9997
Epoch 6/10
11976/11976 [==============================] - 30s 3ms/step - loss: 0.0044 - acc: 0.9992 - val_loss: 0.0014 - val_acc: 0.9997
Epoch 7/10
11976/11976 [==============================] - 30s 3ms/step - loss: 0.0041 - acc: 0.9992 - val_loss: 0.0011 - val_acc:

(1.0, 1.0)